# First steps with ReservoirPy

## Settings

In [1]:
# Name of the config
from config import (
    USE_X_NOISY_DATA,
    USE_Y_NOISY_DATA,
    USE_MIXED_EFFECT,
    INPUT_TO_READOUT,
    READOUT_FEEDBACK_TO_RESERVOIR,
)

print("== Data settings ==")
print(f"\tUSE_X_NOISY_DATA: {USE_X_NOISY_DATA}")
print(f"\tUSE_Y_NOISY_DATA: {USE_Y_NOISY_DATA}")
print(f"\tUSE_MIXED_EFFECT: {USE_MIXED_EFFECT}")

print("== Reservoir settings ==")
print(f"\tINPUT_TO_READOUT: {INPUT_TO_READOUT}")
print(f"\tREADOUT_FEEDBACK_TO_RESERVOIR: {READOUT_FEEDBACK_TO_RESERVOIR}")

#
INIT_SEED = 42
ROOT = "../../../"
TOOL_PATH = ROOT + "tools"
CSV_FILE = ROOT + "data/synthetic_bph_1/simulation.csv"
SERIE_COLUMN_NAME = "individus"
TIMESTEP_COLUMN_NAME = "temps"

== Data settings ==
	USE_X_NOISY_DATA: False
	USE_Y_NOISY_DATA: True
	USE_MIXED_EFFECT: False
== Reservoir settings ==
	INPUT_TO_READOUT: False
	READOUT_FEEDBACK_TO_RESERVOIR: False


## Imports

In [2]:
import sys
import logging
from pathlib import Path

assert Path(TOOL_PATH).exists()
sys.path.append(TOOL_PATH)
sys.path.append("..")  # for model_configurations


from math import ceil
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, QuantileTransformer

# from sklearn.compose import TransformedTargetRegressor
from joblib import dump as jldump
from reservoirpy.nodes import Reservoir, Ridge, Input, ScikitLearnNode

# from reservoirpy.hyper import research, plot_hyperopt_report
from reservoirpy.observables import nrmse, rsquare, mse
import optuna

from model_configurations import (
    ModelConfiguration,
    ScalingData,
)
from time_series_modules import train_test_split_on_series, TimeSerieShapeChanger

/home/francois/miniforge-pypy3/envs/reservoirs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading

In [5]:
data = pd.read_csv(CSV_FILE, sep=";", decimal=",")
data

,individus,temps,x1,x1_obs,x2,x2_obs,x3,x3_obs,x4,x4_obs,...,x5_obs,x6,x6_obs,x7,x7_obs,x8,y_mixed,y_mixed_obs,y_fixed,y_fixed_obs
0,1,0,0.890634,0.941455,-1.008267,-0.988229,0.264669,0.361084,-0.648976,-0.730239,...,0.322978,2.945083,2.900945,-0.744555,-0.744548,0,-1.678414,-1.782819,-2.072956,-2.057110
1,1,1,0.166724,0.704385,-1.164905,-1.226294,0.264059,0.256220,-0.574728,-0.558283,...,0.421475,3.322322,3.421178,-0.631873,-0.623578,0,-1.919671,-1.873649,-2.530125,-2.419892
2,1,2,-0.557185,-0.697930,-1.256533,-1.332657,0.262231,0.185409,-0.507545,-0.526535,...,0.500725,3.722287,3.718409,-0.524238,-0.519302,0,-1.924473,-1.997926,-2.830060,-2.902343
3,1,3,-1.281095,-0.733795,-1.321543,-1.411484,0.259184,0.347365,-0.446756,-0.479332,...,0.561387,4.144980,4.093975,-0.425830,-0.419581,0,-1.803903,-1.854510,-3.062598,-2.978146
4,1,4,-2.005005,-1.866516,-1.371970,-1.376444,0.254919,0.345040,-0.391751,-0.172551,...,0.595408,4.590399,4.583761,-0.339396,-0.340374,0,-1.635467,-1.675288,-3.277499,-3.235019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25495,500,46,-73.726145,-73.652941,0.432651,0.464711,-0.214545,-0.244244,0.266649,0.255834,...,-0.001200,1.476509,1.440325,-1.328121,-1.326202,1,1.739175,1.718363,0.593690,0.507568
25496,500,47,-75.354554,-75.449685,0.428917,0.612389,-0.258311,-0.249547,0.266401,0.164552,...,-0.000893,1.578360,1.587577,-1.332378,-1.333835,1,1.771572,1.809840,0.619710,0.606283
25497,500,48,-76.982963,-76.534494,0.425260,0.403345,-0.303018,-0.279160,0.266177,0.312715,...,0.002469,1.683608,1.674245,-1.336399,-1.329781,1,1.804660,1.964220,0.646286,0.483667
25498,500,49,-78.611371,-79.112450,0.421677,0.315975,-0.348666,-0.344530,0.265974,0.271925,...,-0.003460,1.792253,1.821948,-1.340198,-1.339676,1,1.838422,1.878651,0.673402,0.698870


In [6]:
x_labels = [
    c for c in data.columns if c.startswith("x") and (("_obs" in c) is USE_X_NOISY_DATA)
]
assert len(x_labels) == 8

y_labels = [
    c
    for c in data.columns
    if c.startswith("y")
    and (("_obs" in c) is USE_Y_NOISY_DATA)
    and (("_mixed" in c) is USE_MIXED_EFFECT)
]
assert len(y_labels) == 1

y_true = ["y_mixed" if USE_MIXED_EFFECT else "y_fixed"]

print(x_labels)
print(y_labels)
print(y_true)

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']
['y_fixed_obs']
['y_fixed']


In [77]:
LEN_QUANTILE_GROUPS = 5


def get_quantiles_individuals(df, y_labels, q_ratio, N) -> list[int]:
    assert isinstance(y_labels, list) and len(y_labels) == 1
    y_label = y_labels[0]
    q = df[y_label].quantile(q_ratio, interpolation="nearest")
    print(q)
    ind = df[df[y_label] == q]["individus"]
    assert len(ind) == 1
    ind = ind.iloc[0]
    if N == 1:
        return [ind]
    else:
        next_df = df[df["individus"] != ind]
        return [ind] + get_quantiles_individuals(next_df, y_labels, q_ratio, N - 1)

In [79]:
list_ind_q0 = get_quantiles_individuals(data, y_labels, 0, LEN_QUANTILE_GROUPS)
list_ind_q0

-1017.30947579324
-984.47966897599
-890.97392150911
-631.692389789648
-603.874323961322


[13, 240, 25, 122, 282]

In [80]:
list_ind_q1 = get_quantiles_individuals(data, y_labels, 0.25, LEN_QUANTILE_GROUPS)
list_ind_q1

-25.6653045189547
-25.6152829437283
-25.4770173860149
-25.3780245872982
-25.2564372568844


[384, 216, 220, 108, 1]

In [81]:
list_ind_q2 = get_quantiles_individuals(data, y_labels, 0.5, LEN_QUANTILE_GROUPS)
list_ind_q2

-4.51681609817407
-4.5567618523676
-4.53588563633346
-4.5400075332743
-4.51952094321779


[301, 333, 272, 72, 297]

In [83]:
list_ind_q3 = get_quantiles_individuals(data, y_labels, 0.75, LEN_QUANTILE_GROUPS)
list_ind_q3

0.120700600787735
0.119658904460397
0.114025573965081
0.111105184764508
0.113094025662177


[323, 284, 164, 406, 300]

In [84]:
list_ind_q4 = get_quantiles_individuals(data, y_labels, 1.0, LEN_QUANTILE_GROUPS)
list_ind_q4

406.162478926045
404.267983863392
265.003163198455
259.313986132568
246.130024705664


[103, 101, 145, 269, 265]

### Spliting on series/individuals

In [5]:
TEST_TRAIN_RATIO = 0.2


data_train, data_test = train_test_split_on_series(
    data,
    SERIE_COLUMN_NAME,
    TIMESTEP_COLUMN_NAME,
    test_size=TEST_TRAIN_RATIO,
    random_state=42,
    shuffle=True,
)

Checking the "series x timesteps" hypothesis…
The dataframe has 500 unique series and 51 unique timesteps, for a total of 25500==500x51


In [6]:
data_train

,individus,temps,x1,x1_obs,x2,x2_obs,x3,x3_obs,x4,x4_obs,...,x5_obs,x6,x6_obs,x7,x7_obs,x8,y_mixed,y_mixed_obs,y_fixed,y_fixed_obs
51,2,0,0.917176,1.122408,0.889352,1.014465,1.468332,1.507350,-3.706834,-3.766740,...,0.282201,3.116811,3.179587,-0.732544,-0.727545,1,3.431645,3.588800,1.863642,1.852814
52,2,1,0.544067,1.039673,0.049494,0.106958,1.467464,1.354498,-3.496062,-3.446804,...,0.424360,3.319091,3.250093,-0.500632,-0.493772,1,0.821437,0.943247,0.105767,-0.036786
53,2,2,0.170959,0.373402,-0.441792,-0.334913,1.464858,1.425467,-3.305348,-3.271503,...,0.506399,3.527731,3.540055,-0.310972,-0.306229,1,-0.779822,-0.936180,-0.966104,-0.953343
54,2,3,-0.202149,-0.438238,-0.790365,-0.599801,1.460515,1.478672,-3.132783,-3.260878,...,0.547306,3.742729,3.781823,-0.179771,-0.175969,1,-2.002091,-1.935134,-1.779806,-1.861281
55,2,4,-0.575257,-0.193559,-1.060739,-0.874324,1.454435,1.457878,-2.976639,-3.064382,...,0.557364,3.964087,4.004897,-0.099168,-0.105131,1,-3.056588,-2.889120,-2.479740,-2.412412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25495,500,46,-73.726145,-73.652941,0.432651,0.464711,-0.214545,-0.244244,0.266649,0.255834,...,-0.001200,1.476509,1.440325,-1.328121,-1.326202,1,1.739175,1.718363,0.593690,0.507568
25496,500,47,-75.354554,-75.449685,0.428917,0.612389,-0.258311,-0.249547,0.266401,0.164552,...,-0.000893,1.578360,1.587577,-1.332378,-1.333835,1,1.771572,1.809840,0.619710,0.606283
25497,500,48,-76.982963,-76.534494,0.425260,0.403345,-0.303018,-0.279160,0.266177,0.312715,...,0.002469,1.683608,1.674245,-1.336399,-1.329781,1,1.804660,1.964220,0.646286,0.483667
25498,500,49,-78.611371,-79.112450,0.421677,0.315975,-0.348666,-0.344530,0.265974,0.271925,...,-0.003460,1.792253,1.821948,-1.340198,-1.339676,1,1.838422,1.878651,0.673402,0.698870


In [7]:
data_test

,individus,temps,x1,x1_obs,x2,x2_obs,x3,x3_obs,x4,x4_obs,...,x5_obs,x6,x6_obs,x7,x7_obs,x8,y_mixed,y_mixed_obs,y_fixed,y_fixed_obs
0,1,0,0.890634,0.941455,-1.008267,-0.988229,0.264669,0.361084,-0.648976,-0.730239,...,0.322978,2.945083,2.900945,-7.445547e-01,-0.744548,0,-1.678414,-1.782819,-2.072956,-2.057110
1,1,1,0.166724,0.704385,-1.164905,-1.226294,0.264059,0.256220,-0.574728,-0.558283,...,0.421475,3.322322,3.421178,-6.318732e-01,-0.623578,0,-1.919671,-1.873649,-2.530125,-2.419892
2,1,2,-0.557185,-0.697930,-1.256533,-1.332657,0.262231,0.185409,-0.507545,-0.526535,...,0.500725,3.722287,3.718409,-5.242378e-01,-0.519302,0,-1.924473,-1.997926,-2.830060,-2.902343
3,1,3,-1.281095,-0.733795,-1.321543,-1.411484,0.259184,0.347365,-0.446756,-0.479332,...,0.561387,4.144980,4.093975,-4.258296e-01,-0.419581,0,-1.803903,-1.854510,-3.062598,-2.978146
4,1,4,-2.005005,-1.866516,-1.371970,-1.376444,0.254919,0.345040,-0.391751,-0.172551,...,0.595408,4.590399,4.583761,-3.393963e-01,-0.340374,0,-1.635467,-1.675288,-3.277499,-3.235019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25393,498,46,-9.855542,-9.819291,-2.663384,-2.534075,-0.646690,-0.770152,-0.703329,-0.889543,...,0.000827,1.293258,1.238766,-6.174199e-20,0.000905,1,-1.834571,-1.837263,-2.189698,-2.112023
25394,498,47,-10.071952,-10.017421,-2.672574,-2.645180,-0.696656,-0.625732,-0.702329,-0.672292,...,0.002413,1.228305,1.235071,-2.461226e-20,0.003298,1,-1.746793,-1.852710,-2.079470,-2.012377
25395,498,48,-10.288362,-10.622747,-2.681575,-2.550863,-0.747696,-0.792085,-0.701425,-0.729515,...,-0.001720,1.165025,1.262877,-9.811204e-21,-0.000264,1,-1.660682,-1.532095,-1.971336,-1.930179
25396,498,49,-10.504773,-10.344093,-2.690394,-2.743938,-0.799811,-0.900305,-0.700607,-0.626484,...,0.002232,1.103419,1.110018,-3.911048e-21,0.004990,1,-1.576289,-1.344503,-1.865358,-1.756237


### Data scalings

In [8]:
x_train = data_train[x_labels]
y_train = data_train[y_labels]
x_test = data_test[x_labels]
y_test = data_test[y_true]


scaling_data = {
    "RobustScaler": ScalingData(
        x_train,
        y_train,
        x_test,
        y_test,
        RobustScaler(),
        RobustScaler(),
    ),
    "QuantileTransformer-uniform": ScalingData(
        x_train,
        y_train,
        x_test,
        y_test,
        QuantileTransformer(output_distribution="uniform"),
        QuantileTransformer(output_distribution="uniform"),
    ),
    "QuantileTransformer-normal": ScalingData(
        x_train,
        y_train,
        x_test,
        y_test,
        QuantileTransformer(output_distribution="normal"),
        QuantileTransformer(output_distribution="normal"),
    ),
}

N_train = len(data_train[SERIE_COLUMN_NAME].unique())
T_train = len(data_train[TIMESTEP_COLUMN_NAME].unique())
P_train = len(x_labels)
Y_train = len(y_labels)


N_test = len(data_test[SERIE_COLUMN_NAME].unique())
T_test = len(data_test[TIMESTEP_COLUMN_NAME].unique())
P_test = len(x_labels)
Y_test = len(y_true)

## Training with hyper-parameters optimization

In [9]:
def objective(trial: optuna.Trial) -> float:
    scaling_data_ = scaling_data[
        trial.suggest_categorical(
            "scaler",
            [
                "RobustScaler",
                "QuantileTransformer-normal",
                "QuantileTransformer-uniform",
            ],
        )
    ]

    # example from:  https://reservoirpy.readthedocs.io/en/latest/user_guide/hyper.html#Step-2:-define-the-research-space
    # hyperopt_config = {
    #     "exp": "hyperopt-multiscroll",  # the experimentation name
    #     "hp_max_evals": 200,  # the number of differents sets of parameters hyperopt has to try
    #     "hp_method": "random",  # the method used by hyperopt to chose those sets (see below)
    #     "seed": 42,  # the random state seed, to ensure reproducibility
    #     "instances_per_trial": 5,  # how many random ESN will be tried with each sets of parameters
    #     "hp_space": {  # what are the ranges of parameters explored
    #         "N": ["choice", 500],  # the number of neurons is fixed to 500
    #         "sr": [
    #             "loguniform",
    #             1e-2,
    #             10,
    #         ],  # the spectral radius is log-uniformly distributed between 1e-2 and 10
    #         "lr": ["loguniform", 1e-3, 1],  # idem with the leaking rate, from 1e-3 to 1
    #         "input_scaling": ["choice", 1.0],  # the input scaling is fixed
    #         "ridge": [
    #             "loguniform",
    #             1e-8,
    #             1e1,
    #         ],  # and so is the regularization parameter.
    #         "seed": ["choice", 1234],  # an other random seed for the ESN initialization
    #     },
    # }

    list_rmse = []
    for variable_seed in range(INIT_SEED, INIT_SEED + 3):
        model = ModelConfiguration(
            input_kwargs={},
            reservoir_kwargs={
                "units": trial.suggest_int("N", 100, 1000),
                "sr": trial.suggest_float("sr", 1e-2, 10, log=True),
                "lr": trial.suggest_float("lr", 1e-3, 1, log=True),
                "input_scaling": trial.suggest_float(
                    "input_scaling", 0.5, 2.0, log=False
                ),
                "seed": variable_seed,
            },
            ridge_kwargs={
                "ridge": trial.suggest_float("ridge", 1e-8, 1e1, log=True),
            },
            fit_kwargs={
                "warmup": trial.suggest_int("warmup", 0, 10),
            },
            input_to_readout=INPUT_TO_READOUT,
            readout_feedback_to_reservoir=READOUT_FEEDBACK_TO_RESERVOIR,
        )
        # Train your model and test your model.

        x_train = scaling_data_.x_train.reshape(N_train, T_train, P_train)
        y_train = scaling_data_.y_train.reshape(N_train, T_train, Y_train)
        model.fit(x_train, y_train)

        x_test = scaling_data_.x_test.reshape(N_test, T_test, P_test)
        y_test = scaling_data_.y_test.reshape(N_test, T_test, Y_test)

        y_hat_3D = np.array(model.run(x_test))

        list_rmse.append(nrmse(y_test, y_hat, norm="q1q3"))

    return np.array(list_rmse).mean()

In [11]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "study"
storage_name = "sqlite:///{}.db".format(study_name)

study = optuna.create_study(study_name=study_name, storage=storage_name)
study.optimize(objective, n_trials=100)

[I 2024-10-09 20:44:25,566] A new study created in RDB with name: study


A new study created in RDB with name: study
A new study created in RDB with name: study


[I 2024-10-09 20:44:55,118] Trial 0 finished with value: 19.943975106161865 and parameters: {'scaler': 'QuantileTransformer-uniform', 'N': 406, 'sr': 4.185034421084977, 'lr': 0.009623048164808403, 'input_scaling': 1.0659287970880293, 'ridge': 2.083430810450173e-08, 'warmup': 4}. Best is trial 0 with value: 19.943975106161865.


Trial 0 finished with value: 19.943975106161865 and parameters: {'scaler': 'QuantileTransformer-uniform', 'N': 406, 'sr': 4.185034421084977, 'lr': 0.009623048164808403, 'input_scaling': 1.0659287970880293, 'ridge': 2.083430810450173e-08, 'warmup': 4}. Best is trial 0 with value: 19.943975106161865.
Trial 0 finished with value: 19.943975106161865 and parameters: {'scaler': 'QuantileTransformer-uniform', 'N': 406, 'sr': 4.185034421084977, 'lr': 0.009623048164808403, 'input_scaling': 1.0659287970880293, 'ridge': 2.083430810450173e-08, 'warmup': 4}. Best is trial 0 with value: 19.943975106161865.


[I 2024-10-09 20:45:34,522] Trial 1 finished with value: 1.7489953246124614 and parameters: {'scaler': 'RobustScaler', 'N': 878, 'sr': 0.18845052397959863, 'lr': 0.027431482349623625, 'input_scaling': 0.5319709696917612, 'ridge': 0.024745624255495476, 'warmup': 6}. Best is trial 1 with value: 1.7489953246124614.


Trial 1 finished with value: 1.7489953246124614 and parameters: {'scaler': 'RobustScaler', 'N': 878, 'sr': 0.18845052397959863, 'lr': 0.027431482349623625, 'input_scaling': 0.5319709696917612, 'ridge': 0.024745624255495476, 'warmup': 6}. Best is trial 1 with value: 1.7489953246124614.
Trial 1 finished with value: 1.7489953246124614 and parameters: {'scaler': 'RobustScaler', 'N': 878, 'sr': 0.18845052397959863, 'lr': 0.027431482349623625, 'input_scaling': 0.5319709696917612, 'ridge': 0.024745624255495476, 'warmup': 6}. Best is trial 1 with value: 1.7489953246124614.


[I 2024-10-09 20:46:06,646] Trial 2 finished with value: 2.754262329362341 and parameters: {'scaler': 'RobustScaler', 'N': 611, 'sr': 0.5753603545896345, 'lr': 0.0034106314361387765, 'input_scaling': 0.778220924666521, 'ridge': 1.253398208638458, 'warmup': 8}. Best is trial 1 with value: 1.7489953246124614.


Trial 2 finished with value: 2.754262329362341 and parameters: {'scaler': 'RobustScaler', 'N': 611, 'sr': 0.5753603545896345, 'lr': 0.0034106314361387765, 'input_scaling': 0.778220924666521, 'ridge': 1.253398208638458, 'warmup': 8}. Best is trial 1 with value: 1.7489953246124614.
Trial 2 finished with value: 2.754262329362341 and parameters: {'scaler': 'RobustScaler', 'N': 611, 'sr': 0.5753603545896345, 'lr': 0.0034106314361387765, 'input_scaling': 0.778220924666521, 'ridge': 1.253398208638458, 'warmup': 8}. Best is trial 1 with value: 1.7489953246124614.


## Prediction with best params on train dataset (checking the ability to train)

In [20]:
best_models = []

for variable_seed in range(
    best["seed"], best["seed"] + hyperopt_config["instances_per_trial"]
):

    input_kwargs = {}
    reservoir_kwargs = {
        "units": best["N"],
        "lr": best["lr"],
        "sr": best["sr"],
        "input_scaling": best["input_scaling"],
        "seed": variable_seed,
    }
    ridge_kwargs = {"ridge": best["ridge"]}
    fit_kwargs = {"warmup": best["warmup"]}

    model = ModelConfiguration(
        input_kwargs,
        reservoir_kwargs,
        ridge_kwargs,
        fit_kwargs,
        input_to_readout=best["input_to_readout"],
        readout_feedback_to_reservoir=best["readout_feedback_to_reservoir"],
    )

    model.fit(x_train_3D, y_train_3D)
    best_models.append(model)

In [22]:
list_y_hat_3D = [model.run(x_train_3D) for model in best_models]

In [ ]:
list_rmse = [nrmse(y_train_3D, y_hat_3D) for y_hat_3D in list_y_hat_3D]
list_rmse

In [26]:
idx_best_model = list_rmse.index(min(list_rmse))
y_hat_3D = list_y_hat_3D[idx_best_model]

y_hat_scaled = np.array(y_hat_3D).reshape(
    [N_series_train * N_times_train, len(y_labels)]
)

In [ ]:
N_plots = 15  # un mutliple de 3 c'est mieux…

data_train["y_train_scaled"] = y_train_scaled
data_train["y_hat_scaled"] = y_hat_scaled
series_train = data_train[SERIE_COLUMN_NAME].unique()[:N_plots]


NCOLS = 3
nrows = ceil(len(series_train) / NCOLS)


f, ax = plt.subplots(
    nrows=nrows,
    ncols=NCOLS,
    figsize=(6.5 * NCOLS, 6.5 * nrows),
)


all_y_trained_scaled = data_train[data_train[SERIE_COLUMN_NAME].isin(series_train)][
    "y_train_scaled"
]
y_min = all_y_trained_scaled.min()
y_max = all_y_trained_scaled.max()


for i_serie, serie in enumerate(series_train):
    irow = i_serie // NCOLS
    icol = i_serie % NCOLS
    data_plot = data_train[data_train[SERIE_COLUMN_NAME] == serie]
    ax_ = ax[irow, icol]

    sns.lineplot(
        x=data_plot[TIMESTEP_COLUMN_NAME],
        y=data_plot["y_train_scaled"],
        ax=ax_,
        color="red",
        label="target",
    )
    sns.lineplot(
        x=data_plot[TIMESTEP_COLUMN_NAME],
        y=data_plot["y_hat_scaled"],
        ax=ax_,
        color="blue",
        label="prediction",
    )

    ax_.set_title(f"série #{serie}")
    ax_.legend()
    ax_.set_ymax(ymax)